In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# Step 1: Load and Preprocess Dataset
data = pd.read_csv('lightning_roulette_results_reversed.csv', header=None, names=['number'])

# Convert numbers to integers and remove invalid data
data['number'] = pd.to_numeric(data['number'], errors='coerce')
data = data.dropna(subset=['number'])
data['number'] = data['number'].astype(int)

# Map numbers to dozens, colors, parity, and high/low ranges
def get_dozen(number):
    if 1 <= number <= 12:
        return 0  # Dozen 1
    elif 13 <= number <= 24:
        return 1  # Dozen 2
    elif 25 <= number <= 36:
        return 2  # Dozen 3
    else:
        return 3  # Green (0)

def get_color(number):
    red_numbers = {1, 3, 5, 7, 9, 12, 14, 16, 18, 19, 21, 23, 25, 27, 30, 32, 34, 36}
    if number == 0:
        return "Green"
    elif number in red_numbers:
        return "Red"
    else:
        return "Black"

def get_parity(number):
    return "Even" if number % 2 == 0 else "Odd"

def get_high_low(number):
    return "Low" if 1 <= number <= 18 else "High"

data['dozen'] = data['number'].apply(get_dozen)
data['color'] = data['number'].apply(get_color)
data['parity'] = data['number'].apply(get_parity)
data['high_low'] = data['number'].apply(get_high_low)

# Convert categorical columns to numerical values
color_mapping = {"Red": 1, "Black": 0, "Green": 2}
parity_mapping = {"Even": 1, "Odd": 0}
high_low_mapping = {"Low": 1, "High": 0}

data['color'] = data['color'].map(color_mapping)
data['parity'] = data['parity'].map(parity_mapping)
data['high_low'] = data['high_low'].map(high_low_mapping)

# Step 2: Define the roulette wheel layout
roulette_wheel = [
    0, 32, 15, 19, 4, 21, 2, 25, 17, 34, 6, 27, 13, 36, 11, 30, 8, 23, 10, 5, 24, 16, 33, 1, 20, 14, 31, 9, 22, 18, 29, 7, 28, 12, 35, 3, 26
]

# Step 3: Create a dictionary for fast access to neighbors
wheel_neighbors = {}
for i, number in enumerate(roulette_wheel):
    neighbors = [roulette_wheel[(i-1) % len(roulette_wheel)], roulette_wheel[(i+1) % len(roulette_wheel)]]
    wheel_neighbors[number] = neighbors

# Step 4: Define the sections based on the Voisins, Tiers, and Orphelins sets
voisins = {22, 18, 29, 7, 28, 12, 35, 3, 26, 0, 32, 15, 19, 4, 21, 2, 25}
tiers = {27, 13, 36, 11, 30, 8, 23, 10, 5, 24, 16, 33}
orphelins = {17, 34, 6, 1, 20, 14, 31, 9}

def get_section(number):
    if number in voisins:
        return "Voisins"
    elif number in tiers:
        return "Tiers"
    elif number in orphelins:
        return "Orphelins"
    else:
        return "Green"  # For 0, it's part of the "Green" section

# Step 5: Apply new features to the dataframe
data['wheel_neighbors_1'] = data['number'].apply(lambda x: wheel_neighbors[x][0])
data['wheel_neighbors_2'] = data['number'].apply(lambda x: wheel_neighbors[x][1])
data['section'] = data['number'].apply(get_section)

# Step 6: Map the 'section' to numerical values
section_mapping = {"Voisins": 0, "Tiers": 1, "Orphelins": 2, "Green": 3}
data['section'] = data['section'].map(section_mapping)

# Step 7: Add the 'column' feature (1, 2, or 3)
def get_column(number):
    if number == 0:
        return 0  # For number 0, set column as 0
    else:
        return (number - 1) % 3 + 1

data['column'] = data['number'].apply(get_column)

# Step 8: Map the 'column' to numerical values (mapping already returns 1, 2, or 3, so no additional mapping needed)



# Step 9: Create lag features (last 10 spins as input sequence)
def create_lag_features(df, lag_count):
    lagged_data = []
    target = []
    for i in range(len(df) - lag_count):
        lagged_data.append(df[['number', 'color', 'parity', 'high_low', 'wheel_neighbors_1', 'wheel_neighbors_2', 'section', 'column']].iloc[i:i+lag_count].values)
        target.append(df['dozen'].iloc[i+lag_count])
    return np.array(lagged_data), np.array(target)

lag_count = 10  # Use the last 10 spins as input features
X, y = create_lag_features(data, lag_count)

# One-hot encode target (dozen)
y = to_categorical(y, num_classes=4)  # 4 classes: Dozen 1, Dozen 2, Dozen 3, Green

# Step 10: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 11: Build Improved LSTM Model
model = Sequential([
    LSTM(128, input_shape=(lag_count, 8), return_sequences=True, activation='relu'),  # Updated for 8 features
    Dropout(0.3),
    LSTM(64, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # 4 output classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 12: Train the Model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 8)  # Update for 8 features
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 8)

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Step 13: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

# Step 14: Save the Model
model.save('improved_roulette_lstm_model_with_column.h5')


Epoch 1/100


C:\Users\aajit\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1355/1355 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.3285 - loss: 1.2245 - val_accuracy: 0.3177 - val_loss: 1.2013
Epoch 2/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3243 - loss: 1.1925 - val_accuracy: 0.3174 - val_loss: 1.2000
Epoch 3/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3285 - loss: 1.1924 - val_accuracy: 0.3172 - val_loss: 1.2007
Epoch 4/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3306 - loss: 1.1916 - val_accuracy: 0.3172 - val_loss: 1.2008
Epoch 5/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3292 - loss: 1.1922 - val_accuracy: 0.3177 - val_loss: 1.2019
Epoch 6/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3309 - loss: 1.1932 - val_accuracy: 0.3286 - val_loss: 1.2003
Epoch 7/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3240 - loss: 1.1902 - val_accuracy: 0.3172 - val_loss: 1.2007
Epoch 8/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3291 - loss: 1.1903 - val

Test Accuracy: 0.32
